In [23]:
import pathlib

folder_path = pathlib.Path("./fixtures/tests/join_vtp")
output_path = pathlib.Path("./fixtures/tests/join_vtp/combined_polydata.vtp")

file_names = [
    "lanternim.vtp",
    "marquise_xm_ym.vtp",
    "marquise_xm_yp.vtp",
    "marquise_xp_ym.vtp",
    "marquise_xp_yp.vtp",
]

In [24]:
import vtk
from cfdmod.api.vtk.write_vtk import read_polydata
import math

for i, file_n in enumerate(file_names):
    file_path = folder_path / file_n
    polydata = read_polydata(file_path)

    if i == 0:
        combined_polydata = vtk.vtkPolyData()
        combined_polydata.DeepCopy(polydata)
        locator = vtk.vtkCellLocator()
        locator.SetDataSet(combined_polydata)
        locator.BuildLocator()
        continue

    cell_data = combined_polydata.GetCellData()
    cell_array_names = [cell_data.GetArrayName(i) for i in range(cell_data.GetNumberOfArrays())]

    for scalar_lbl in cell_array_names:
        # Identify cells with real values
        real_value_cells1 = set(
            cell_id
            for cell_id in range(combined_polydata.GetNumberOfCells())
            if not math.isnan(
                combined_polydata.GetCellData().GetArray(scalar_lbl).GetValue(cell_id)
            )
        )

        real_value_cells2 = set(
            cell_id
            for cell_id in range(polydata.GetNumberOfCells())
            if not math.isnan(polydata.GetCellData().GetArray(scalar_lbl).GetValue(cell_id))
        )

        # Merge real-value cells
        for cell_id in real_value_cells2:
            cell = polydata.GetCell(cell_id)
            cell_center = [0.0, 0.0, 0.0]
            for i in range(cell.GetNumberOfPoints()):
                cell_center = [sum(x) for x in zip(cell_center, cell.GetPoints().GetPoint(i))]
            cell_center = [x / cell.GetNumberOfPoints() for x in cell_center]

            cell_id1 = locator.FindCell(cell_center)

            if cell_id1 != -1:
                value = polydata.GetCellData().GetArray(scalar_lbl).GetValue(cell_id)
                combined_polydata.GetCellData().GetArray(scalar_lbl).SetValue(cell_id1, value)


# Save the final polydata if needed
writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName(output_path)
writer.SetInputData(combined_polydata)
writer.Write()

1